# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [25]:
# Load the data
pd.set_option('max_columns', None)
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)


,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,next_pymnt_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
23702,20000.0,0.0819,628.49,RENT,98000.0,Not Verified,Feb-2019,low_risk,n,13.15,0.0,2.0,18.0,1.0,19676.0,27.0,w,19508.01,19508.01,619.39,619.39,491.99,127.40,0.0,0.0,0.0,628.49,Apr-2019,0.0,1.0,Individual,0.0,1427.0,46884.0,2.0,3.0,0.0,1.0,22.0,27208.0,78.0,3.0,6.0,6726.0,56.0,49800.0,1.0,1.0,2.0,7.0,2605.0,21446.0,33.2,0.0,0.0,187.0,380.0,2.0,2.0,0.0,2.0,2.0,0.0,6.0,9.0,7.0,9.0,6.0,15.0,21.0,9.0,18.0,0.0,0.0,0.0,3.0,100.0,0.0,1.0,0.0,84510.0,46884.0,32100.0,34710.0,N,N
46065,6600.0,0.1180,218.59,RENT,35000.0,Source Verified,Jan-2019,low_risk,n,20.51,1.0,0.0,10.0,0.0,4496.0,25.0,w,6291.11,6291.11,432.85,432.85,308.89,123.96,0.0,0.0,0.0,218.59,Apr-2019,1.0,1.0,Individual,0.0,182.0,56751.0,0.0,3.0,1.0,1.0,8.0,52255.0,104.0,0.0,0.0,2142.0,77.0,23800.0,0.0,0.0,1.0,1.0,5675.0,9358.0,18.6,0.0,0.0,107.0,110.0,26.0,8.0,0.0,87.0,8.0,0.0,1.0,6.0,1.0,1.0,18.0,7.0,7.0,6.0,10.0,0.0,0.0,0.0,1.0,91.7,0.0,0.0,0.0,73933.0,56751.0,11500.0,50133.0,N,N
65412,13000.0,0.1298,295.66,MORTGAGE,132000.0,Verified,Jan-2019,low_risk,n,7.15,0.0,1.0,18.0,0.0,11180.0,33.0,w,12529.82,12529.82,868.23,868.23,470.18,398.05,0.0,0.0,0.0,295.66,May-2019,0.0,1.0,Individual,0.0,0.0,337973.0,2.0,1.0,1.0,2.0,5.0,11394.0,95.0,1.0,4.0,4279.0,16.0,127000.0,3.0,0.0,1.0,6.0,18776.0,109387.0,4.5,0.0,0.0,122.0,427.0,1.0,1.0,6.0,1.0,5.0,1.0,6.0,7.0,13.0,13.0,8.0,16.0,19.0,7.0,18.0,0.0,0.0,0.0,2.0,93.9,0.0,0.0,0.0,480004.0,22574.0,114600.0,12000.0,N,N
41498,20000.0,0.0702,617.73,MORTGAGE,135000.0,Verified,Jan-2019,low_risk,n,19.21,0.0,1.0,19.0,0.0,175165.0,32.0,w,18995.61,18995.61,1258.86,1258.86,1004.39,254.47,0.0,0.0,0.0,617.73,May-2019,0.0,1.0,Individual,0.0,0.0,609170.0,1.0,2.0,3.0,4.0,2.0,25742.0,80.0,1.0,1.0,16466.0,61.0,271600.0,2.0,1.0,3.0,5.0,32062.0,80928.0,57.5,0.0,0.0,109.0,152.0,9.0,2.0,3.0,9.0,2.0,0.0,12.0,14.0,14.0,17.0,6.0,16.0,23.0,14.0,19.0,0.0,0.0,0.0,4.0,100.0,7.1,0.0,0.0,769350.0,200907.0,190600.0,32000.0,N,N
57017,6500.0,0.1691,231.46,MORTGAGE,34776.0,Not Verified,Jan-2019,low_risk,n,20.67,0.0,0.0,8.0,1.0,7820.0,15.0,w,6074.53,6074.53,688.27,688.27,425.47,262.80,0.0,0.0,0.0,231.46,May-2019,0.0,1.0,Individual,0.0,0.0,23537.0,0.0,2.0,0.0,0.0,33.0,15717.0,47.0,0.0,0.0,6919.0,56.0,8600.0,1.0,0.0,1.0,0.0,4707.0,640.0,92.2,0.0,0.0,140.0,331.0,31.0,31.0,1.0,31.0,8.0,0.0,2.0,3.0,5.0,6.0,3.0,6.0,11.0,3.0,8.0,0.0,0.0,0.0,0.0,100.0,50.0,1.0,0.0,42290.0,23537.0,8200.0,33690.0,N,N
46779,18500.0,0.0819,581.35,RENT,45000.0,Not Verified,Jan-2019,low_risk,n,24.69,0.0,1.0,7.0,0.0,9220.0,13.0,w,17586.71,17586.71,1183.75,1183.75,913.29,270.46,0.0,0.0,0.0,581.35,Apr-2019,0.0,1.0,Individual,0.0,0.0,25415.0,1.0,2.0,0.0,2.0,18.0,16195.0,66.0,1.0,1.0,7650.0,64.0,67800.0,4.0,2.0,1.0,3.0,3631.0,56194.0,75.0,0.0,0.0,173.0,200.0,3.0,3.0,0.0,3.0,4.0,0.0,2.0,3.0,3.0

In [32]:
# Preview the data
df.sample(10)

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,next_pymnt_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
39646,4200.0,0.1131,138.13,MORTGAGE,33258.0,Not Verified,Feb-2019,low_risk,n,26.71,0.0,0.0,11.0,0.0,4731.0,16.0,f,4001.98,4001.98,270.98,270.98,198.02,72.96,0.0,0.0,0.0,138.13,May-2019,0.0,1.0,Individual,0.0,0.0,23517.0,0.0,2.0,0.0,1.0,16.0,18786.0,73.0,0.0,3.0,964.0,57.0,15600.0,1.0,1.0,2.0,4.0,2138.0,5815.0,24.5,0.0,0.0,143.0,388.0,13.0,13.0,0.0,13.0,3.0,0.0,3.0,8.0,4.0,5.0,4.0,9.0,12.0,8.0,11.0,0.0,0.0,0.0,0.0,93.8,50.0,0.0,0.0,41214.0,23517.0,7700.0,25614.0,N,N
22733,7500.0,0.0819,235.69,MORTGAGE,61000.0,Source Verified,Feb-2019,low_risk,n,30.26,0.0,0.0,13.0,0.0,13418.0,17.0,w,7315.50,7315.50,232.28,232.28,184.50,47.78,0.0,0.0,0.0,235.69,Apr-2019,0.0,1.0,Individual,0.0,0.0,253412.0,1.0,4.0,0.0,1.0,13.0,16943.0,48.0,1.0,2.0,8406.0,43.0,35600.0,1.0,3.0,4.0,4.0,19493.0,21882.0,38.0,0.0,0.0,27.0,64.0,5.0,5.0,1.0,5.0,2.0,0.0,4.0,4.0,7.0,7.0,4.0,8.0,12.0,4.0,13.0,0.0,0.0,0.0,2.0,100.0,28.6,0.0,0.0,295758.0,30361.0,35300.0,35008.0,N,N
10846,16000.0,0.1557,385.45,MORTGAGE,125000.0,Source Verified,Mar-2019,low_risk,n,19.42,0.0,2.0,27.0,1.0,16637.0,83.0,w,15822.15,15822.15,371.61,371.61,177.85,193.76,0.0,0.0,0.0,385.45,May-2019,0.0,1.0,Individual,0.0,0.0,258951.0,3.0,4.0,4.0,8.0,3.0,105238.0,94.0,5.0,7.0,3866.0,54.0,72500.0,9.0,11.0,16.0,15.0,9960.0,20835.0,34.5,0.0,0.0,130.0,277.0,6.0,3.0,8.0,11.0,3.0,2.0,5.0,12.0,7.0,16.0,31.0,22.0,44.0,12.0,26.0,0.0,0.0,0.0,9.0,98.6,14.3,1.0,0.0,320104.0,121875.0,31800.0,103416.0,N,N
22623,6250.0,0.1797,225.86,RENT,80000.0,Not Verified,Feb-2019,low_risk,n,9.62,0.0,1.0,9.0,0.0,5707.0,14.0,w,6117.73,6117.73,219.62,219.62,132.27,87.35,0.0,0.0,0.0,225.86,Apr-2019,0.0,1.0,Individual,0.0,0.0,30984.0,0.0,3.0,0.0,0.0,35.0,25277.0,73.0,1.0,1.0,2003.0,72.0,8300.0,0.0,3.0,3.0,1.0,3443.0,1289.0,63.2,0.0,0.0,78.0,56.0,10.0,10.0,0.0,10.0,0.0,1.0,2.0,3.0,3.0,3.0,5.0,6.0,8.0,3.0,9.0,0.0,0.0,0.0,1.0,71.4,33.3,0.0,0.0,43052.0,30984.0,3500.0,34752.0,N,N
64723,17625.0,0.2235,490.30,MORTGAGE,77000.0,Not Verified,Jan-2019,low_risk,n,35.10,0.0,0.0,26.0,0.0,44689.0,36.0,w,17215.70,17215.70,1449.02,1449.02,409.30,1039.72,0.0,0.0,0.0,490.30,May-2019,0.0,1.0,Individual,0.0,0.0,190696.0,0.0,2.0,0.0,0.0,43.0,22662.0,47.0,0.0,2.0,4673.0,56.0,72500.0,0.0,0.0,0.0,2.0,7334.0,19759.0,59.5,0.0,0.0,131.0,311.0,16.0,16.0,1.0,26.0,16.0,0.0,12.0,21.0,14.0,17.0,4.0,23.0,30.0,21.0,26.0,0.0,0.0,0.0,0.0,100.0,71.4,0.0,0.0,257294.0,67351.0,48800.0,48312.0,N,N
49105,28000.0,0.0819,570.29,OWN,80000.0,Not Verified,Jan-2019,low_risk,n,22.71,0.0,0.0,10.0,0.0,33345.0,26.0,w,27239.03,27239.03,1127.84,1127.84,760.97,366.87,0.0,0.0,0.0,570.29,Apr-2019,0.0,1.0,Individual,0.0,0.0,44744.0,1.0,3.0,1.0,1.0,7.0,11399.0,63.0,1.0,1.0,19043.0,68.0,48100.0,0.0,0.0,1.0,2.0,4474.0,11447.0,72.1,0.0,0.0,183.0,189.0,5.0,5.0,1.0,93.0,7.0,0.0,3

In [30]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [6]:
 df.shape

(68817, 86)

In [7]:
 df = df.dropna(axis='columns', how='all')

In [8]:
df.shape

(68817, 86)

In [9]:
df = df.dropna()

In [10]:
df.shape

(68817, 86)

## Split the Data into Training and Testing

In [33]:
# Create our features
# X_scaler wasn't working because home_ownership had 3 values - couldn't go binary
# X = df.drop(columns='loan_status')
X = pd.get_dummies(df.drop(columns='loan_status'))

# Create our target
targ = df['loan_status'].copy()
y = targ

In [34]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,policy_code,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0,68817.000000,68817.000000,68817.0,68817.0,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.00000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,24.666347,16018.845833,16015.919519,970.639473,970.431151,658.748750,311.855687,0.035072,0.0,0.0,517.202218,0.016188,1.0,0.0,188.664298,1.634976e+05,1.023047,3.290481,0.843730,1.909644,14.303152,4.324044e+04,69.21191,1.240042,2.622245,6297.491710,55.702472,42431.895360,1.382071,1.687795,2.189023,4.827572,14142.936193,16991.852667,48.330667,0.006830,2.347065,125.468605,175.136841,14.294215,7.179781,1.441882,24.714954,7.688028,0.472485,3.806123,5.648038,5.135156,7.414403,9.500153,8.630818,13.517663,5.653036,12.571995,0.0,0.0,0.052138,2.219423,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,0.009285,0.526309,0.106747,0.357659,0.478007,0.373992,0.148001,0.371696,0.451066,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,12.318917,10002.695495,10004.985940,1229.563301,1229.569864,1126.237377,279.786954,1.086720,0.0,0.0,816.021053,0.134774,0.0,0.0,1808.457195,1.723699e+05,1.176782,2.965579,0.970543,1.544994,13.972619,4.956130e+04,22.33711,1.471745,2.502608,5702.281757,18.495480,37166.229316,1.543220

In [35]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [36]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [37]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [38]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [39]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [45]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier, EasyEnsembleClassifier
model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)

#fit model
model.fit(X_train_scaled, y_train)

#predict
y_predict = model.predict(X_test_scaled)

In [46]:
# Calculated the balanced accuracy score
print(f'BAS: {balanced_accuracy_score(y_test,y_predict)}')

BAS: 0.7887512850910909


In [47]:
# Display the confusion matrix
confusion_matrix(y_test, y_predict)

array([[   71,    30],
       [ 2146, 14958]])

In [48]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_predict))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.70      0.87      0.06      0.78      0.60       101
   low_risk       1.00      0.87      0.70      0.93      0.78      0.63     17104

avg / total       0.99      0.87      0.70      0.93      0.78      0.63     17205



In [50]:
# List the features sorted in descending order by feature importance
# sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)
sorted(zip(model.feature_importances_, X.columns), reverse=True)[:10]

[(0.07876809003486353, 'total_rec_prncp'),
 (0.05883806887524815, 'total_pymnt'),
 (0.05625613759225244, 'total_pymnt_inv'),
 (0.05355513093134745, 'total_rec_int'),
 (0.0500331813446525, 'last_pymnt_amnt'),
 (0.02966959508700077, 'int_rate'),
 (0.021129125328012987, 'issue_d_Jan-2019'),
 (0.01980242888931366, 'installment'),
 (0.01747062730041245, 'dti'),
 (0.016858293184471483, 'out_prncp_inv')]

### Easy Ensemble Classifier

In [56]:
# Train the Classifier
easy_model = EasyEnsembleClassifier(n_estimators=100, random_state=1)

# Fit the model
easy_model.fit(X_train_scaled, y_train)

# Make the predictions
eas_y_pred = easy_model.predict(X_test_scaled)

In [57]:
# Calculated the balanced accuracy score
print(f'Easy E BAS: {balanced_accuracy_score(y_test, eas_y_pred)}')

Easy E BAS: 0.931601605553446


In [58]:
# Display the confusion matrix
confusion_matrix(y_test, eas_y_pred)

array([[   93,     8],
       [  985, 16119]])

In [59]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, eas_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.09      0.92      0.94      0.16      0.93      0.87       101
   low_risk       1.00      0.94      0.92      0.97      0.93      0.87     17104

avg / total       0.99      0.94      0.92      0.97      0.93      0.87     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    Easy Ensemble had the best balanced accuracy score.

2. Which model had the best recall score?

    The Easy Ensemble model had a high risk recall of .92 and low risk recall score of .94, which is much better than the Random Forrest. 

3. Which model had the best geometric mean score?

    Easy Enemble again was much better than the Random Forrest GM score. 

4. What are the top three features?

    Total received principle, total payment, and total payment inv, which probably means total payments invoiced.